We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

### THE DATA

In [1]:
import pandas as pd

In [2]:
census=pd.read_csv('census_data.csv')

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def fixed_label(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket']=census['income_bracket'].apply(fixed_label)

In [9]:
#could have also hav done this as well


#census['income_bracket'].apply(lambda label:  int(label==' <=50K'))

### Perform a Train Test Split on the Data

In [7]:
from sklearn.model_selection import train_test_split

In [9]:
x_data=census.drop('income_bracket',axis=1)

y_labels=census['income_bracket']

X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

### Create the Feature Columns for tf.esitmator


In [11]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [12]:
import tensorflow as tf

In [13]:
#these are all th categorical features
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [14]:
#these are are the numerical values
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [15]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

### Create Input Function


In [16]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,
                                              batch_size=100,num_epochs=None,
                                              shuffle=True)

In [17]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_eval_distribute': None, '_master': '', '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_model_dir': 'C:\\Users\\Jag\\AppData\\Local\\Temp\\tmpbjj8y3wr', '_save_summary_steps': 100, '_device_fn': None, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_task_id': 0, '_task_type': 'worker', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_is_chief': True, '_protocol': None, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002871135B518>, '_evaluation_master': '', '_experimental_distribute': None, '_train_distribute': None, '_num_worker_replicas': 1, '_service': None, '_session_creation_timeout_secs': 7200, '_save_checkpoints_secs': 600, '_experimental_max_worker_delay_secs': None}


In [18]:
model.train(input_fn=input_func,steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Jag\AppDat

INFO:tensorflow:global_step/sec: 132.782
INFO:tensorflow:loss = 33.11964, step = 3901 (0.744 sec)
INFO:tensorflow:global_step/sec: 125.881
INFO:tensorflow:loss = 54.307705, step = 4001 (0.796 sec)
INFO:tensorflow:global_step/sec: 127.931
INFO:tensorflow:loss = 36.04114, step = 4101 (0.783 sec)
INFO:tensorflow:global_step/sec: 126.08
INFO:tensorflow:loss = 51.430813, step = 4201 (0.791 sec)
INFO:tensorflow:global_step/sec: 120.724
INFO:tensorflow:loss = 98.2755, step = 4301 (0.838 sec)
INFO:tensorflow:global_step/sec: 135.809
INFO:tensorflow:loss = 370.64124, step = 4401 (0.726 sec)
INFO:tensorflow:global_step/sec: 123.164
INFO:tensorflow:loss = 42.831474, step = 4501 (0.815 sec)
INFO:tensorflow:global_step/sec: 133.427
INFO:tensorflow:loss = 32.922752, step = 4601 (0.746 sec)
INFO:tensorflow:global_step/sec: 119.428
INFO:tensorflow:loss = 44.065613, step = 4701 (0.843 sec)
INFO:tensorflow:global_step/sec: 132.039
INFO:tensorflow:loss = 115.33856, step = 4801 (0.753 sec)
INFO:tensorflow

### Evaluation



In [20]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=len(X_test),
      shuffle=False)

In [21]:
prediction=list(model.predict(input_fn=eval_input_func))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Jag\AppData\Local\Temp\tmpbjj8y3wr\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [23]:
prediction[0]

{'all_class_ids': array([0, 1]),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.21008086], dtype=float32),
 'logits': array([-1.3244381], dtype=float32),
 'probabilities': array([0.78991914, 0.21008085], dtype=float32)}

In [25]:
final_pred=[]

for pred in prediction:
    final_pred.append(pred['class_ids'][0])

In [26]:
final_pred[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [27]:
from sklearn.metrics import classification_report

In [29]:
print(classification_report(y_test,final_pred))

             precision    recall  f1-score   support

          0       0.87      0.92      0.89      7436
          1       0.68      0.56      0.62      2333

avg / total       0.82      0.83      0.83      9769

